# Imports + Path Specification

In [ ]:
import sys
import os
import torch
from torch import optim
from time import time

In [ ]:
tile2vec_dir = '/home/agupta21/gcloud/231n_gitproject'
sys.path.append('../')
sys.path.append(tile2vec_dir)

In [ ]:
from src.datasets import TileTripletsDataset, GetBands, RandomFlipAndRotate, ClipAndScale, ToFloatTensor, triplet_dataloader
from src.tilenet import make_tilenet

In [ ]:
from src.training import prep_triplets, train_triplet_epoch

In [ ]:
import numpy as np
from torch.autograd import Variable
import sys
from src.resnet import ResNet18

In [ ]:
#Random Forest Classification

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Dataloader + TileNet Setup

In [ ]:
# Environment stuff
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda = torch.cuda.is_available()
print(cuda)

True


In [ ]:
# Defining Parameters
img_type = 'naip'
tile_dir = '/home/agupta21/gcloud/231n_gitproject/data/triplets/'
bands = 4
augment = False
batch_size = 50
shuffle = True
num_workers = 4
n_triplets = 108700 #modification: adding 8700 more for semi-supervised learning purposes

In [ ]:
dataloader = triplet_dataloader(img_type, tile_dir, bands=bands, augment=augment,
                                batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, 
                                n_triplets=n_triplets, pairs_only=True)
print('Dataloader set up complete.')

Dataloader set up complete.


In [ ]:
in_channels = bands
z_dim = 512

In [ ]:
TileNet = make_tilenet(in_channels=in_channels, z_dim=z_dim, strat2=False)
TileNet.train()
if cuda: TileNet.cuda()
print('TileNet set up complete.')

TileNet set up complete.


In [ ]:
#Hyperparameter 
lr = 0.0003 #hyper tuned
optimizer = optim.Adam(TileNet.parameters(), lr=lr, betas=(0.9, 0.999)) #hyper tuned

# Training Model + Writing Each Epoch to File

In [ ]:
epochs = 10
margin = 10
l2 = 0.01
print_every = 10000
save_models = True

In [ ]:
model_dir = '/home/agupta21/gcloud/231n_gitproject/models/'
if not os.path.exists(model_dir): os.makedirs(model_dir)

In [ ]:
t0 = time()
#with open(results_fn, 'w') as file:

plot_list = []
print('Begin training.................')
for epoch in range(0, epochs):
    (avg_loss, bullshit, avg_l_n, avg_l_d, avg_l_nd) = train_triplet_epoch(
        TileNet, cuda, dataloader, optimizer, epoch+1, margin=margin, l2=l2,
        print_every=print_every, t0=t0)
    plot_list.append((epoch+1,avg_loss))
    append_name = "strat1ht1_ep" + str(epoch+1) + ".ckpt"
    if save_models:
        model_fn = os.path.join(model_dir,append_name)
        torch.save(TileNet.state_dict(),model_fn)

Begin training.................
Epoch 1: [10000/108700 (9%)], Avg loss: 6.0657
Epoch 1: [20000/108700 (18%)], Avg loss: 5.3830
Epoch 1: [30000/108700 (28%)], Avg loss: 5.0873
Epoch 1: [50000/108700 (46%)], Avg loss: 4.9285
Epoch 1: [60000/108700 (55%)], Avg loss: 4.7964
Epoch 1: [70000/108700 (64%)], Avg loss: 4.8594
Epoch 1: [80000/108700 (74%)], Avg loss: 4.6677
Epoch 1: [90000/108700 (83%)], Avg loss: 4.7307
Epoch 1: [100000/108700 (92%)], Avg loss: 4.6602
Epoch 2: [10000/108700 (9%)], Avg loss: 4.6709
Epoch 2: [20000/108700 (18%)], Avg loss: 4.6138
Epoch 2: [30000/108700 (28%)], Avg loss: 4.5226
Epoch 2: [40000/108700 (37%)], Avg loss: 4.5351
Epoch 2: [50000/108700 (46%)], Avg loss: 4.5406
Epoch 2: [60000/108700 (55%)], Avg loss: 4.5161
Epoch 2: [70000/108700 (64%)], Avg loss: 4.4106
Epoch 2: [80000/108700 (74%)], Avg loss: 4.4889
Epoch 2: [90000/108700 (83%)], Avg loss: 4.4245
Epoch 2: [100000/108700 (92%)], Avg loss: 4.4215
Finished epoch 2: 5658.949s
  Average loss: 4.5061
  Ave

In [ ]:
print(plot_list)


# Pre Process Y Values

In [ ]:
#note to self: need to embed tiles first and then figure out loop for reading in the saved model and plotting
#the classification accuracies per epoch
tile_dir = '../data/tiles'
n_tiles = 1000
y = np.load(os.path.join(tile_dir, 'y.npy'))
print(y.shape)
#print(set(y))

In [ ]:
# Check CDL classes
print(set(y))

In [ ]:
y = LabelEncoder().fit_transform(y)
print(set(y))

# Load in Epochs' Weights + Run each on tile embeddings + Random Forest 

In [ ]:
#set up model
in_channels = 4
z_dim = 512
cuda = torch.cuda.is_available()

In [ ]:
def getTileEmbeddings(tilenet):
    X = np.zeros((n_tiles, z_dim))
    for idx in range(n_tiles):
        tile = np.load(os.path.join(tile_dir, '{}tile.npy'.format(idx+1)))
        # Get first 4 NAIP channels (5th is CDL mask)
        tile = tile[:,:,:4]
        # Rearrange to PyTorch order
        tile = np.moveaxis(tile, -1, 0)
        tile = np.expand_dims(tile, axis=0)
        # Scale to [0, 1]
        tile = tile / 255
        # Embed tile
        tile = torch.from_numpy(tile).float()
        tile = Variable(tile)
        if cuda: tile = tile.cuda()
        z = tilenet.encode(tile)
        if cuda: z = z.cpu()
        z = z.data.numpy() #1 by 512
        X[idx,:] = z
    return X

In [ ]:
epochsToAccuracy = []
epochsToSTD = []
for i in range(0,epochs): #iterator
    curEpoch = i + 1
    # Setting up model
    tilenet = ResNet18()
    if cuda: tilenet.cuda()
    model_fn = "../models/strat1ht1_ep"+str(curEpoch)+".ckpt" #open file
    #checkpoint = torch.load(model_fn)
    tilenet.load_state_dict(torch.load(model_fn), strict=False)
    #tilenet.load_state_dict(checkpoint)
    tilenet.eval()
    
    X = getTileEmbeddings(tilenet) #function above
    
    #train random forest classifier
    n_trials = 100
    accs = np.zeros((n_trials,))
    for i in range(n_trials):
        # Splitting data and training RF classifer
        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)
        rf = RandomForestClassifier(n_estimators=100)
        rf.fit(X_tr, y_tr) #X-tr is 512 by 1
        accs[i] = rf.score(X_te, y_te)
    print("Results for Epoch Number: ", str(curEpoch))
    print('Mean accuracy: {:0.4f}'.format(accs.mean()))
    print('Standard deviation: {:0.4f}'.format(accs.std()))
    epochsToAccuracy.append((curEpoch,accs.mean()))
    epochsToSTD.append((curEpoch,accs.std()))
print(epochsToAccuracy)
print(epochsToSTD)
    #save value
    
    
    
    

In [ ]:
print(epochsToSTD)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(*zip(*epochsToAccuracy))
plt.title('Triplet Augmentation Average Test Performance')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()
print(epochsToAccuracy[9][1], "was the Max Accuracy")

In [ ]:
plt.plot(*zip(*plot_list))
plt.title('Triplet Augmentation Loss Performance')
plt.xlabel('Epochs')
plt.ylabel('Loss')
print("The min loss was",str(plot_list[9][1]))
plt.show()

In [ ]:
plt.plot(*zip(*epochsToSTD))
plt.title('STD on Random Forest Test Prediction')
plt.xlabel('Epochs')
plt.ylabel('Standard Deviation')
plt.show()